<a href="https://colab.research.google.com/github/rishav119/demo1/blob/mybranch/Samourai_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heuristic to capture the Samourai Wallet Trasactions
[Samourai](https://samouraiwallet.com/) 

# Setting up Required Connections

## Connecting with BQ Client

In [1]:
PROJECT = 'bigquery-public-data'
GCS_BUCKET_NAME  = 'rishav_bucket'
BILLING_PROJECT='intelligence-team'
DATASET = 'crypto_bitcoin'

In [2]:
def show_table(table_name, num_rows=10):
  SQL = f"""
    SELECT * 
    FROM 
      `{table_name}`
    LIMIT {num_rows}
  """
  df = pd.io.gbq.read_gbq(SQL, project_id=BILLING_PROJECT, verbose=True, dialect='standard')
  return df.head()

def run_query(SQL, no_limit=False, limit=100):
  if not no_limit:
    _sql = ' '.join([SQL, f'LIMIT {limit}'])
  else:
    _sql = SQL

  df = pd.io.gbq.read_gbq(_sql, project_id=BILLING_PROJECT, verbose=True, dialect='standard')

  if not no_limit:
    num_rows = limit
  else:
    num_rows = len(df)
  return df.head(num_rows)

## Authenticating with Google Account

In [3]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Importing Required Packages

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Confirmed Samourai CoinJoin Transactions

We are taking the list of confirmed Samourai coinjoin transactions from a [Github](https://github.com/nopara73/WasabiVsSamourai/blob/master/WasabiVsSamourai/SamouraiCoinJoins.txt) Page created Nopara. 

In [5]:
#Samourai Transactions on BQ
SQL = f"""
        SELECT 
            trxn_hash
        FROM 
            `intelligence-team.rishav.samourai_trxns`
        GROUP BY 1
"""
conf_samourai_trxns=run_query(SQL, no_limit=True)
conf_samourai_trxns.head()

,trxn_hash
0,72024630a91bcd48162664af1f5cf26f1ad5629aeb7f81...
1,f73c5ec27f76396eac945c184fe22480e22a5ac419b9d8...
2,e8ec0557daa37502e0ad37d74cdbad0b323b8daf1d262e...
3,9317fad60a5bed6f54b62b4cda3c98e7238ec8ecfb8d9f...
4,1ddd4e49dcee946dc76efa18cd807fa33d471ea38f31e1...


In [6]:
#Total confirmed samourai conjoin transctions
len(conf_samourai_trxns)

7310

## Analyzing Samourai transactions

### Finding the samourai reused change addresses

In [7]:
SQL = f"""
          SELECT 
              ARRAY_TO_STRING(outputs.addresses,'') as output_address, 
              COUNT(DISTINCT transaction_hash) as trxn_count
          FROM 
              `bigquery-public-data.crypto_bitcoin.outputs` AS outputs
          WHERE 
              transaction_hash IN 
                        (
                            SELECT 
                                trxn_hash 
                            FROM 
                                `intelligence-team.rishav.samourai_trxns`
                        )
          GROUP BY 1
          ORDER BY 2 DESC
"""
samourai_outputs_addr = run_query(SQL, no_limit=True)
samourai_outputs_addr.head()

,output_address,trxn_count
0,bc1qrm76lr2plkcw6m3r5ng8y7rcwyls9h0q2ferv0,1
1,bc1q65qm85x4azrdxherzalp9ufcaafm8cuj3ydx6w,1
2,bc1qsgggr2xhy5v2hx2a7hlen822w0qdz2cch4vlm8,1
3,bc1qq9jvu9c386eyqqvlj9x8c2mc80r8ahavgmausq,1
4,bc1qqxptz7en8hx8fuzp9yauznwe3qaq6x4sxghaw9,1


Unlike Wasabi wallet, Samourai wallet is not resuing any change address

### Last transaction date of the confirmed Samourai transactions

In [14]:
#Finding the last active date from the list of conformed samourai transactions
SQL = f"""
          SELECT MAX(dt) as max_trxn_date
          FROM (
                  SELECT 
                      txn.hash as trxn_hash, DATE(block_timestamp) as dt
                  FROM 
                      `bigquery-public-data.crypto_bitcoin.transactions` AS txn
                  WHERE 
                      txn.hash IN (SELECT trxn_hash FROM `intelligence-team.rishav.samourai_trxns`)
                  GROUP BY 1,2    
          )
"""
conf_samourai_trxns_max_date = run_query(SQL, no_limit=True)
conf_samourai_trxns_max_date

,max_trxn_date
0,2020-01-05


So from the list of conformed samourai trxns which has 7310 trxns, the last transaction date is 2020-01-05.

# Preparing Custom Heuristic to find potentail samourai transactions
Rules used:
1. Bech32 addresses used in both inputs and outputs side
2. Output address and input address count is exactly = 5
3. For all the 5 output addresses in a given trxn the outputs.value in BTC is either of these (0.05, 0.25, 2.5)
4. Samourai wallet was started after 2015-01-01.
6. Fee/byte is less than 15



In [12]:
if input = 'Incremental':
  SAMOURAI_START_DATE = '2015-01-01'
else:
  SAMOURAI_START_DATE = "Today's date"
OUTPUT_VALUE = (0.05, 0.25, 2.5)
FEE_SIZE = 15
ADDRESS_COUNT = 5
SQL = f"""
            WITH segwit_tx_outputs AS (
              SELECT
              transactions.hash AS tx
            , MAX(output_count) AS output_count
            , SUM(CASE WHEN ARRAY_TO_STRING(outputs.addresses, "") LIKE "bc1%" THEN 1 ELSE 0 END) num_segwit_outputs
              FROM
              `bigquery-public-data.crypto_bitcoin.transactions` AS transactions,
              UNNEST(transactions.outputs) AS outputs
              WHERE
                DATE(block_timestamp_month) >= '{SAMOURAI_START_DATE}'
              AND input_count= {ADDRESS_COUNT}
              AND output_count= {ADDRESS_COUNT}
              AND fee/size<= {FEE_SIZE}
              AND output_value/1e8 IN {OUTPUT_VALUE}
              GROUP BY 1
          ),
          segwit_tx_inputs AS (
            SELECT
              transactions.hash AS tx
            , MAX(input_count) AS input_count
            , SUM(CASE WHEN ARRAY_TO_STRING(inputs.addresses, "") LIKE "bc1%" THEN 1 ELSE 0 END) num_segwit_inputs
            FROM
              `bigquery-public-data.crypto_bitcoin.transactions` AS transactions,
              UNNEST(transactions.inputs) AS inputs
            WHERE
                DATE(block_timestamp_month) >= '{SAMOURAI_START_DATE}'
              AND input_count= {ADDRESS_COUNT}
              AND output_count= {ADDRESS_COUNT}
              AND fee/size<= {FEE_SIZE}
              AND output_value/1e8 IN {OUTPUT_VALUE}
            GROUP BY 1
          ),

          samourai_txns as (
            SELECT
              segwit_tx_outputs.tx as txn
            FROM
              segwit_tx_outputs
            INNER JOIN
              segwit_tx_inputs
            ON
              segwit_tx_outputs.tx = segwit_tx_inputs.tx
            WHERE 
              num_segwit_outputs = output_count AND 
              num_segwit_inputs = input_count
            GROUP BY 1
          ),

          address_list as (
            SELECT 
            ARRAY_TO_STRING(inputs.addresses,'') as address
            FROM 
              `staging-btc-etl.crypto_bitcoin.inputs_view` AS inputs
            WHERE 
              transaction_hash IN (SELECT txn FROM samourai_txns)
            AND DATE(block_timestamp_month) >= '{SAMOURAI_START_DATE}'
            GROUP BY 1

            UNION DISTINCT

            SELECT 
              ARRAY_TO_STRING(outputs.addresses,'') as address
            FROM 
              `staging-btc-etl.crypto_bitcoin.outputs_view` AS outputs
            WHERE 
              transaction_hash IN (SELECT txn FROM samourai_txns)
            AND DATE(block_timestamp_month) >= '{SAMOURAI_START_DATE}'
            GROUP BY 1
        ),
      samourai_final_address_list as
        (
          SELECT 
            addresses 
          FROM 
              `intelligence-team.clusters.bitcoin`
          WHERE 
              cluster_id IN
                (
                    SELECT 
                      cluster_id
                    FROM
                      `intelligence-team.clusters.bitcoin` 
                    WHERE
                      addresses IN 
                            (
                              SELECT 
                                address 
                              FROM 
                                address_list
                            )
                    GROUP BY 1
                    HAVING 
                      COUNT(DISTINCT addresses) = 5
                )
          GROUP BY 1
        )
        SELECT txn FROM samourai_txns
        GROUP BY 1
    

"""
samourai_txns = run_query(SQL, no_limit=True)
samourai_txns.head()

,txn
0,78536f22f0ce0d722d352c961d5c23cde4b90979a1bbab...
1,1d6486dba1b019d09c2a80ed1d4fc29df8f676b109e9c7...
2,a7c4eab3de10202470df57c70ec9c34ebc0366eeee53c9...
3,f38a180540d93c126617ea37224131b8d1fa8306587d2b...
4,8ff8684717921932894bd46971bdfd24afce44e1b0f75a...


In [13]:
# Count of total clustered samourai transactions
len(samourai_txns)

72615

### Calculating the total Clustered Samourai Addresses

In [ ]:
SQL = f"""
          SELECT 
              address
          FROM 
            `intelligence-team.rishav.samourai_clustered_addresses`  
    """
samourai_clustered_addresses = run_query(SQL, no_limit=True)
samourai_clustered_addresses.head()

,address
0,bc1q2f5fyfv29zzzzvtxclqlu0q0m8pfmrka3f8r2l
1,bc1qgmxtq6tyvfk36wrg73g7uq3ty2tepn2fk5x3zw
2,bc1qvjhtk2xz45gve7k08qfksyuyg7mql9tkmez59u
3,bc1q8n5txl048fmlx9edadkt9pwa605l03fxx2gryt
4,bc1ql5pc266sx97nmayyh673l9p4ehkqwxuj6pmx2g


### Count of total clustered Samourai addresses

In [ ]:
# Count of total samourai addresses
len(samourai_clustered_addresses)

359915

# Sanity Checks

## Cluster Size check
Checking the cluster size of the clustered samourai addresses

In [ ]:
SQL = f"""
          SELECT cluster_id, count(distinct addresses) as address_count ,min(addresses) as address
          FROM `intelligence-team.clusters.bitcoin` 
          WHERE cluster_id IN (SELECT cluster_id FROM `intelligence-team.clusters.bitcoin` WHERE addresses IN (SELECT address FROM `intelligence-team.rishav.samourai_clustered_addresses`))
          GROUP BY 1
          ORDER BY 2 DESC
          LIMIT 200
    """
samourai_addresses_cluster = run_query(SQL, no_limit=True)
samourai_addresses_cluster

,cluster_id,address_count,address
0,93575479997565,294,bc1q0ch8xydpcsxgq57udmkn3qlqj99ugr7pyc0j3c
1,93501999998072,159,bc1q0696v0amkk8uggtwl0tca5jhxtgrxv2j2cw3m5
2,93128589999452,99,bc1q00gwe7at5f7yc0qzf40w2a3a2pmrn83j9j9wxv
3,93522419999952,86,bc1q03538xjwxdt4m639xljzye6gqd6dvy8ggvd3nx
4,93518509997949,81,bc1q0ttga8vr0slv8dc0m9us9k6f43mkf3a3u9jqam
...,...,...,...
195,93801709999927,9,bc1q3a858dkxy7qthr2vfnm5fmuuupqwp7jw5dlwq4
196,93520429999414,9,bc1q7409jpnt6cczl9le9zej0dtwpewpelrvsc7vr5
197,93562539999157,9,bc1q47h65qn7j7k4vr28eh5spvj20pnm2v7knk7hyt
198,93396869999456,9,bc1q096jfththu4n9zmmtsqay5j8lljx5dtzw4rhat


In [ ]:
SQL = f"""
          SELECT address_count, COUNT(DISTINCT cluster_id) as clusterId_count
          FROM
            (
                SELECT cluster_id, count(distinct addresses) as address_count ,min(addresses) as address
                FROM `intelligence-team.clusters.bitcoin` 
                WHERE cluster_id IN (SELECT cluster_id FROM `intelligence-team.clusters.bitcoin` WHERE addresses IN (SELECT address FROM `intelligence-team.rishav.samourai_clustered_addresses`))
                GROUP BY 1
            )
          GROUP BY 1
          ORDER BY 1 DESC
    """
samourai_addresses_cluster_check = run_query(SQL, no_limit=True)
samourai_addresses_cluster_check

,address_count,clusterId_count
0,294,1
1,159,1
2,99,1
3,86,1
4,81,1
5,67,1
6,47,1
7,39,1
8,38,2
9,32,1


We got only one cluster of size 294 addresses, most (>98%) of the cluster size is of 5 which is expected.

## Duplicity Check
Checking if any of the clustered wasabi addresses is already tagged in our database

In [ ]:
SQL = f"""
          SELECT 
            tag_name_verbose, addresses 
          FROM 
            `intelligence-team.bitcoin_flat.cluster_poing_tagging_view`
          WHERE 
            addresses IN 
                    (
                      SELECT 
                          address
                      FROM 
                        `intelligence-team.rishav.samourai_clustered_addresses`
                    )
          ORDER BY 1
    """
samourai_duplicate_addresses = run_query(SQL, no_limit=True)
samourai_duplicate_addresses

,tag_name_verbose,addresses


No addresses from clustered samourai addresses were found to be duplicate/tagged in our database.

## Precision
Of all clustered samourai transaction, what proportion were actual samourai transaction

In [9]:
## Total confirmed samourai transaction we have is
conf_samourai_trxns_count= len(conf_samourai_trxns)
conf_samourai_trxns_count

7310

Ok, so we got a total of 7310 transactions which are confirmed samourai transaction and the max date for those list of confirmed trxns is 2020-01-05

Now we need to filter out our clustered transactions till Jan 5th 2020.

In [10]:
SQL = f"""
          SELECT 
              COUNT(DISTINCT txn.hash)
          FROM 
              `bigquery-public-data.crypto_bitcoin.transactions` AS txn
          WHERE 
              txn.hash IN (SELECT txn FROM `intelligence-team.rishav.samourai_clustered_trxns_fee`)
          AND 
            DATE(block_timestamp) <= '2020-01-05'
    """
samouri_cluster_trxns_jan20_count = run_query(SQL, no_limit=True)
samouri_cluster_trxns_jan20_count

,f0_
0,7789


In [11]:
samouri_cluster_trxns_jan20_count = int(samouri_cluster_trxns_jan20_count['f0_'])

In [13]:
samouri_cluster_trxns_jan20_count

7789

In [14]:
SQL = f"""
            with clustered_samouri_trxns as
                  (
                            SELECT 
                                txn.hash as transaction_hash
                            FROM 
                                `bigquery-public-data.crypto_bitcoin.transactions` AS txn
                            WHERE 
                                txn.hash IN (SELECT txn FROM `intelligence-team.rishav.samourai_clustered_trxns_fee`)
                            AND 
                              DATE(block_timestamp) <= '2020-01-05'
                  )

            SELECT 
                COUNT(DISTINCT transaction_hash)
            FROM 
                clustered_samouri_trxns
            WHERE 
                transaction_hash IN 
                          (
                              SELECT 
                                  trxn_hash
                              FROM
                                  `intelligence-team.rishav.samourai_trxns`
                              GROUP BY 1   
                        )
    """
samourai_cluster_trxns_jan20_precision_count = run_query(SQL, no_limit=True)
samourai_cluster_trxns_jan20_precision_count = int(samourai_cluster_trxns_jan20_precision_count['f0_'])
samourai_cluster_trxns_jan20_precision_count

7310

In [15]:
#Calculating Precision
Precision = round(samourai_cluster_trxns_jan20_precision_count/samouri_cluster_trxns_jan20_count,4)
Precision

0.9385

## Recall
of all confirmed samourai transactions, what proportion did we actaully clustered 

In [16]:
#Calculating Recall
Recall = round(samourai_cluster_trxns_jan20_precision_count/len(conf_samourai_trxns),4)
Recall

1.0